In [1]:
#  A best_model.pth file is created in the directory after the training is done. We'll use this to run the robot now

import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [2]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [3]:
device = torch.device('cuda')
model = model.to(device)

#load the files onto the GPU from the CPU

In [4]:
# we need to create a preprocessing function as the data from the camera isnt the same as the data we obtained from training


# we convert BGR to RGB
# we convert HCW layout CHW layout
# scale the image( camera provides values from [0,255] and our classifier [0,1] so we need to scale it down)
# transfer from CPU memory to GPU memory
# Add a batch dimension

import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [5]:
# Displays the probability of the robot being blocked through a slider

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

Error loading module `ublox_gps`: No module named 'serial'


In [6]:
#create a robot instance so we can drive the motors

from jetbot import Robot

robot = Robot()